# SMS Spam Filter

The goal of this project is to **build a spam filter for SMS messages.**

To be able to do it we need to achive the following four steps. We need to make sure that the computer:
1. Learns how humans classify messages.
2. Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
3. Classifies a new message based on these probability values. If the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

Our first task is to **"teach" the computer how to classify messages**. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

**DATASET**
The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) or directy from [this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where find some of the authors' papers can be found.

## Exploration

In [1]:
import pandas as pd
import numpy as np

In [2]:
sms_spam = pd.read_csv('SMSSpamCollection', 
                      sep='\t', 
                      header=None,
                      names = ['Label','SMS'])

In [3]:
#explore how many of each there are
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

There are 13.4% spam and 86.6% non-spam ('ham') messages in the dataset.

In [4]:
sms_spam.shape

(5572, 2)

## Preparation: Test definition
Once our spam filter is done, we'll **need to test how good it is** with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

- **A training set**, which we'll use to "train" the computer how to classify messages.
- **A test set**, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep **80% of our dataset for training**, and **20% for testing** (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

We're going to start by randomizing the entire dataset to ensure that spam and ham messages are spread properly throughout the dataset.

In [5]:
#randomisethe whole set
randomised = sms_spam.sample(frac=1, random_state=1)

#select 80% for training
training = randomised.sample(frac=0.8)
training.shape

(4458, 2)

In [6]:
# subtract training fromthe full df to create a test set
#using concat
test = pd.concat([randomised,training]).drop_duplicates(keep=False)
test.shape

(970, 2)

Expected result was 1114, unsure why there were more duplicates here that got dropped.

In [7]:
# checking if the regular samplin returns correct result
test_2 = randomised.sample(frac=0.2)
test_2.shape

(1114, 2)

In [8]:
#generating test set using difference
test_3 = randomised.loc[randomised.index.difference(training.index)]
test_3.shape

(1114, 2)

In [9]:
#generating test set using disjunctive union
test_4 = randomised.loc[randomised.index ^  training.index]
test_4.shape

(1114, 2)

Both return the correct number of rows.

In [10]:
test_set = test_4

#check % of spam/ham in both sets
test_percent = test_set['Label'].value_counts(normalize=True)
training_percent = training['Label'].value_counts(normalize=True)

In [11]:
#turn into dfs to compare 
df1 = test_percent.rename_axis('label').reset_index(name='percent_test')
df2 = training_percent.rename_axis('label').reset_index(name='Percent')

In [12]:
df1['percent_training'] = df2['Percent']

#add original percentages to compare with tes & training
original_percent = sms_spam['Label'].value_counts(normalize=True).rename_axis('label').reset_index(name='percent_original')
df1['percent_original'] = original_percent['percent_original']
df1

,label,percent_test,percent_training,percent_original
0,ham,0.868941,0.865186,0.865937
1,spam,0.131059,0.134814,0.134063


There are 13.4% spam and 86.6% non-spam ('ham') messages in the original dataset. The values in the test and training datasets are similar: spam 12% and 13.3%, non spam 88% and 86.7% respectively.

## Data Cleaning
To bring the data in a format that will allow us to extract easily all the information we need, we'll need to perform data cleaning.

We'll standardise the messages in the SMS column and then transform it so that each word becomes its own column. The index will then indicate hw many times a given word occurs in each spam/ham message.

First we remove the punctuation and bring all the words to lower case.

In [13]:
#convert into string
#standardise case
training['SMS'] = training['SMS'].astype(str).str.lower()

#remove all unwanted characters
import re
training['SMS'].replace('\W',' ',regex=True, inplace = True)

#check if worked
training['SMS'].head()

4102    gsoh  good with spam the ladies u could b a ma...
2857    japanese proverb  if one can do it  u too can ...
3238    ron say fri leh  n he said ding tai feng cant ...
5239          jay wants to work out first  how s 4 sound 
5506    god s love has no limit  god s grace has no me...
Name: SMS, dtype: object

### Create vocabulary

In [14]:
#set of unique words is a vocabulary
#create a vocab from all words in the SMS col

#split sms into lists
training['SMS'] = training['SMS'].astype(str).str.split()

In [15]:
vocabulary = []
for row in training['SMS']:
    for word in row:
        vocabulary.append(word)

len(vocabulary)

72429

In [16]:
#transform into a set to remove duplicates
vocabulary = set(vocabulary)

In [17]:
#transform back into list
vocabulary = list(vocabulary)

len(vocabulary)

7757

In [18]:
#create a dictonary with occurence of each word per row

#create a dictionary with all words, where:
    #each key is a unique word (a string) from the vocabulary, and 
    #each value is a list of the length of training set, 
    #each element in the list is a 0

word_count_per_sms = {unique_word: [0] * len(training['SMS']) for unique_word in vocabulary}

In [19]:
for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_count_per_sms[word][index] +=1

In [20]:
wc_sms = pd.DataFrame(word_count_per_sms)

wc_sms.head()

,nearly,yuo,ve,w1jhl,50p,wine,punishment,missed,difference,using,...,80122300p,strips,chloe,mike,sweatter,cheetos,others,dey,yan,stopbcm
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
wc_sms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4458 entries, 0 to 4457
Columns: 7757 entries, nearly to stopbcm
dtypes: int64(7757)
memory usage: 263.8 MB


In [22]:
#check for nans
wc_sms.isnull().values.any()

False

**No NaNs in the dataframe.**

In [23]:
training.head()

,Label,SMS
4102,spam,"[gsoh, good, with, spam, the, ladies, u, could..."
2857,ham,"[japanese, proverb, if, one, can, do, it, u, t..."
3238,ham,"[ron, say, fri, leh, n, he, said, ding, tai, f..."
5239,ham,"[jay, wants, to, work, out, first, how, s, 4, ..."
5506,ham,"[god, s, love, has, no, limit, god, s, grace, ..."


In [24]:
#Note: attempting to use concat function caused the kernel to die (multiple tries)
#using join instead
joined = training.join(wc_sms, how='left')
joined.head()

,Label,SMS,nearly,yuo,ve,w1jhl,50p,wine,punishment,missed,...,80122300p,strips,chloe,mike,sweatter,cheetos,others,dey,yan,stopbcm
4102,spam,"[gsoh, good, with, spam, the, ladies, u, could...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2857,ham,"[japanese, proverb, if, one, can, do, it, u, t...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3238,ham,"[ron, say, fri, leh, n, he, said, ding, tai, f...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5239,ham,"[jay, wants, to, work, out, first, how, s, 4, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5506,ham,"[god, s, love, has, no, limit, god, s, grace, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
joined.isnull().values.any()

True

Suddenly there are NaNs and integers have been turned into floats. 

In [26]:
joined.isnull().sum()

Label        0
SMS          0
nearly     897
yuo        897
ve         897
          ... 
cheetos    897
others     897
dey        897
yan        897
stopbcm    897
Length: 7759, dtype: int64

In [27]:
wc_training = pd.concat([training,wc_sms], axis=1)
wc_training.head()

,Label,SMS,nearly,yuo,ve,w1jhl,50p,wine,punishment,missed,...,80122300p,strips,chloe,mike,sweatter,cheetos,others,dey,yan,stopbcm
0,ham,"[go, until, jurong, point, crazy, available, o...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ham,"[ok, lar, joking, wif, u, oni]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
#check for nans
wc_training.isnull().values.any()

True

In [29]:
wc_training['Label'].value_counts(dropna=False)

ham     3857
NaN      897
spam     601
Name: Label, dtype: int64

Since the NaNs seem to be in the label & sms fields, we can just drop them.

In [30]:
wc_training = wc_training.dropna(subset=['Label'])

In [31]:
wc_training.isnull().values.any()

True

In [32]:
wc_training['Label'].isnull().value_counts()

False    4458
Name: Label, dtype: int64

In [33]:
wc_training['tea'].isna().value_counts()

False    3561
True      897
Name: tea, dtype: int64

**this section not useful in the end, but kept for reference**

Will try to do the same but reset index first.

In [34]:
t_no_index = training.reset_index(drop=True)
wc_no_index = wc_training.reset_index(drop=True)

In [35]:
new_concat = pd.concat([t_no_index, wc_no_index], axis=1)

In [36]:
new_concat.head(3)

,Label,SMS,Label,SMS,nearly,yuo,ve,w1jhl,50p,wine,...,80122300p,strips,chloe,mike,sweatter,cheetos,others,dey,yan,stopbcm
0,spam,"[gsoh, good, with, spam, the, ladies, u, could...",ham,"[go, until, jurong, point, crazy, available, o...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ham,"[japanese, proverb, if, one, can, do, it, u, t...",ham,"[ok, lar, joking, wif, u, oni]",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ham,"[ron, say, fri, leh, n, he, said, ding, tai, f...",ham,"[u, dun, say, so, early, hor, u, c, already, t...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
new_concat.isnull().values.any()

True

In [38]:
joined['you'].value_counts(dropna=False)

0.0    2542
NaN     897
1.0     739
2.0     177
3.0      66
4.0      16
5.0      15
6.0       3
7.0       2
8.0       1
Name: you, dtype: int64

However we join the databases there are stll 906 NaNs. We will remove them.

In [39]:
wc_training.dropna(inplace=True)

In [40]:
wc_training.isna().values.any()

False

## Calculate probabilities

We'll calculate all the parameters using the equations below:

\\[P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}\\]

\\[P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}\\]

To calculate P(wi|Spam) and P(wi|Ham) we need to classify the new messages using these formulas:

\\[P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\]
\\[P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)\\]

We'll first calculate:

* P(Spam), P(Ham)
* n_spam, n_ham, n_vocabulary

Notes:
- *n_spam* is equal to the number of words in all the spam messages — it's not equal to the number of spam messages, and it's not equal to the total number of unique words in spam messages.

- *n_ham* is equal to the number of words in all the non-spam messages — it's not equal to the number of non-spam messages, and it's not equal to the total number of unique words in non-spam messages.

In [41]:
#P(Spam)
p_spam = (len(wc_training[wc_training['Label']=='spam']) / len(wc_training)) 
p_spam_100 = (len(wc_training[wc_training['Label']=='spam']) / len(wc_training))*100

#P(Ham)
p_ham = 1 - p_spam
p_ham_100 = 100 - p_spam_100

print("The probability that a message is a spam one is around", round(p_spam_100,2), 
      "%, and the probability that it's ham is around", round(p_ham_100,2), "%.")

The probability that a message is a spam one is around 13.42 %, and the probability that it's ham is around 86.58 %.


In [42]:
print(p_spam, p_ham)

0.13423195731536086 0.8657680426846391


In [43]:
wc_training['words_sum'] = wc_training['SMS'].apply(len)
wc_training.head()

,Label,SMS,nearly,yuo,ve,w1jhl,50p,wine,punishment,missed,...,strips,chloe,mike,sweatter,cheetos,others,dey,yan,stopbcm,words_sum
0,ham,"[go, until, jurong, point, crazy, available, o...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
1,ham,"[ok, lar, joking, wif, u, oni]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14
5,spam,"[freemsg, hey, there, darling, it, s, been, 3,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36


In [44]:
wc_training['words_sum'].isna().value_counts()

False    3561
Name: words_sum, dtype: int64

In [45]:
#all words in spam messages
spam_messages = wc_training[wc_training['Label']=='spam']
n_spam = spam_messages['words_sum'].sum()

#all words in ham messages
ham_messages = wc_training[wc_training['Label']=='ham']
n_ham = ham_messages['words_sum'].sum()

In [46]:
#vocabulary
n_vocabulary = len(vocabulary)

#laplace smoothing
alpha = 1

For each word, we need to calculate both P(wi|Spam) and P(wi|Ham).

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities.

We'll initialize two dictionaries, where each key-value pair is a unique word (from the vocabulary) represented as a string, and the value is 0. One dictionary will store the parameters for P(wi|Spam), and the other for P(wi|Ham).

In [47]:
spam_dictionary = {}
ham_dictionary = {}

for word in vocabulary:
    spam_dictionary[word] = [0]
    ham_dictionary[word] = [0]

In [48]:
#n_wi_spam = number of times a word occurs in all spam messages
#n_wi_not_spam = number of times a word occurs in all spam messages

for word in vocabulary:
    n_wi_spam = spam_messages[word].sum()
    n_wi_not_spam = ham_messages[word].sum()
    
    p_wi_spam = (n_wi_spam + alpha) / (n_spam + alpha*n_vocabulary)
    spam_dictionary[word] = p_wi_spam
    
    p_wi_not_spam = (n_wi_not_spam + alpha) / (n_ham + alpha*n_vocabulary)
    ham_dictionary[word] = p_wi_not_spam


## Creating a filter
with all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as input a new message (w1, w2, ..., wn)
* Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
* Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    * If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    * If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    *  If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [73]:
import re

def classify(message):

    #the input is string, clean the message
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    #calculate probabilities
    
    #initiate variables, priors
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # foreach word find correspnding values in the dictionaries
    #update final probability
    for word in message:
        if word in spam_dictionary:
            p_spam_given_message *= spam_dictionary[word]
        if word in ham_dictionary:
            p_ham_given_message *= ham_dictionary[word]
        

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    #print results!
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [66]:
#test funciton
m_1 = 'WIN PRIZE!! MEGA secret code to unlock the money: 12X34.'
classify(m_1)

P(Spam|message): 9.800860367363035e-28
P(Ham|message): 2.8982188281301195e-25
Label: Ham


Unsure why it labels it `ham` where it's clearly spam! Everything is `ham`! :facepalm:

In [124]:
m_3 = 'WINNER!! This is the secret code to unlock the money money money: C3421.'
classify(m_3)

m_4 = 'Sounds good, Tom, then see u there'
classify(m_4)

P(Spam|message): 8.892447588598952e-35
P(Ham|message): 3.0097341913610213e-31
Label: Ham
P(Spam|message): 1.7324403485976313e-23
P(Ham|message): 2.2862854593486107e-21
Label: Ham


In [70]:
m_2 = 'Wanna grab dinner tomorrow?'
classify(m_2)

P(Spam|message): 7.397541643016726e-17
P(Ham|message): 7.458644494635064e-15
Label: Ham


## Classify the test set

We'll try to determine how well the spam filter does on our test set of 1,114 messages.

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human).

We'll change the `classify()` function that we wrote previously to return the labels instead of printing them. Below, note that we now have return statements instead of `print()` functions.

In [79]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in spam_dictionary:
            p_spam_given_message *= spam_dictionary[word]

        if word in ham_dictionary:
            p_ham_given_message *= ham_dictionary[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [84]:
#create a column in the test set with the predicitons
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set['predicted'].value_counts(normalize=True)

ham                           0.995512
spam                          0.002693
needs human classification    0.001795
Name: predicted, dtype: float64

We can now compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

**Accuracy = correctly classified messages / total number of messages**


In [117]:
correct = 0
total = test_set.shape[0]

for index, row in test_set.iterrows():
    if row['predicted'] == row['Label']:
        correct += 1

accuracy = correct/ total
accuracy_round = round((accuracy*100),2)
print('The filter is',accuracy_round, '% accurate.')
print('Correctly classified messages:', correct, '(', accuracy_round, '%)'"\n"
     'Incorrectly classified messsages:', total - correct, '(', round(100-accuracy_round,2), '%)'
     )

The filter is 86.62 % accurate.
Correctly classified messages: 965 ( 86.62 %)
Incorrectly classified messsages: 149 ( 13.38 %)


## Conclusion & Next steps
**We initially aimed for an accuracy of over 80%, the filter is 86.62% accurate which is better than expected.**

Next steps we can take to increase the accuracy of this filer:
* Isolate the 149 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
* Make the filtering process more complex by making the algorithm sensitive to letter case.